## Atividade proposta

Você foi contratado para desenvolver um sistema simples de gerenciamento de livros em uma
biblioteca. O sistema deve permitir cadastrar novos livros, listar todos os livros disponíveis, buscar
um livro pelo título, e gerar um gráfico com a quantidade de livros por gênero.

### Passo 1: Definir a classe Livro
- Comece definindo a estrutura básica de um livro usando uma classe em Python. Cada livro
terá atributos como título, autor, gênero e quantidade disponível.
### Passo 2: Criar a lista de livros
- Inicialize uma lista vazia para armazenar os livros que serão cadastrados.

### Passo 3: Implementar funções para gerenciar os livros
- Função para cadastrar um novo livro
- Função para listar todos os livros
- Função para buscar um livro pelo título

### Passo 4: Utilizar a biblioteca Matplotlib para gerar um gráfico
- Instalação da Matplotlib
- Gerar o gráfico de quantidade de livros por gênero

### Passo 5: Testar o sistema

<hr />

#### Classe Genero

In [ ]:
from curses.ascii import isdigit
from typing import Callable

from attr.filters import include
from matplotlib import ticker


def normalize_book_genre_name(genre_name):
    genre_name = genre_name.lower()

    return genre_name

class BookGenre:
    def __init__(self, genre: str):
        self.name: str = genre.strip()
        self.normalized: str = normalize_book_genre_name(self.name)

#### Classe Livro

In [ ]:
class Book:
    def __init__(self, title: str, author: str, genre: BookGenre, quantity: int = 1):
        self.title = title
        self.author = author
        self.genre = genre
        self.quantity = quantity

    def is_available(self) -> bool:
        return self.quantity > 0


def validate_book(book: Book):
    if not book.genre:
        raise Exception(f"Genero é obrigatório: {book.genre}")

    if not book.title:
        raise Exception(f"Titulo é obrigatório")

    if not book.author:
        raise Exception(f"Autor é obrigatório")


### Classe BookCollection
A classe vai está resposavel por adicionar livros que pertence a mesmo genero

In [ ]:
from functools import reduce
import typing
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import matplotlib.cm as cm
import numpy as np

TypeBookSet = typing.Set[Book]

class CollectionBook:
    def __init__(self):
        self.books_by_genres: dict[str, TypeBookSet] = dict()

    def genre_exists(self, genre_name: str) -> bool:
        return genre_name in self.books_by_genres

    def upsert_book(self, book: Book):
        book_genre_name_normalized: str = book.genre.name
        if book_genre_name_normalized not in self.books_by_genres.keys():
            self.books_by_genres[book_genre_name_normalized] = set()

        for book_on_list in self.books_by_genres[book_genre_name_normalized]:
            if book_on_list.title == book.title:
                raise Exception("Livro já existente")

        self.books_by_genres[book_genre_name_normalized].add(book)

    def add_book(self, book: Book):
        validate_book(book)
        self.upsert_book(book)

    def list_genres(self):
        return list(self.books_by_genres.keys())

    def list_all_books(self) -> list[Book]:
        all_books: list[Book] = list()
        for book_set in list(self.books_by_genres.values()):
            for books_of_set in book_set:
                all_books.append(books_of_set)

        return all_books

    def list_books_by_genre(self, genre_name: str):
        if self.genre_exists(genre_name):
            return list()

        return self.books_by_genres.values()

    def total_books(self):
        return len(self.list_all_books())

    def total_genres(self):
        return len(self.list_genres())

    def search_book_by_title(self, title: str):
        return list(filter(lambda book: title in book.title, self.list_all_books()))

    def report_all_books_by_genre(self):
        genres: list[str] = list()
        books_quantity: list[int] = list()
        books_by_genre = self.books_by_genres.items()
        quantity_genre_to_list: int = 5

        for (genre, books_of_genre) in books_by_genre:
            if len(genres) >= quantity_genre_to_list:
                genres.append("Outros")
                sum_total_books_by_list_genre_actual = reduce(lambda sum, actual_quantity: sum + actual_quantity, books_quantity, 0)

                sum_others_books: int = abs(self.total_books() - sum_total_books_by_list_genre_actual)

                books_quantity.append(sum_others_books)
                break
            genres.append(genre)
            books_quantity.append(len(books_of_genre))

        return genres, books_quantity

    def show_chart_about_books_by_genres(self):
        genres, books_by_genre = self.report_all_books_by_genre()
        fig, ax = plt.subplots(figsize=(12, 6))

        colors = cm.tab20(np.linspace(0.3, 1, len(genres)))

        ax.bar(genres, books_by_genre, label=genres, color=colors)


        ax.yaxis.set_major_locator(mticker.MaxNLocator(integer=True))

        ax.set_xlabel("Genero")
        ax.set_ylabel("Quantidade de livros")

        ax.set_title("Quantidade de livros por genero")

        plt.show()

    def show_chart_difference_between_total_of_books_and_genres(self):
        total_genres = self.total_genres()
        total_books = self.total_books()
        data: list[int] = list([total_genres, total_books])
        legends_labels: list[str] = list(["Generos", "Livros"])
        fig, ax = plt.subplots(figsize=(12, 6))

        def func(pct, allvals):
            absolute = int(np.round(pct/100.*np.sum(allvals)))
            return f"{pct:.1f}%\n({absolute:d} livros)"


        wedges, texts, autotexts = ax.pie(data, autopct=lambda pct: func(pct, data),
                                          textprops=dict(color="w"))

        ax.legend(wedges, legends_labels,
                  loc="center left",
                  bbox_to_anchor=(1, 0, 0.5, 1))

        plt.setp(autotexts, size=8, weight="bold")

        ax.set_title("Livros vs Genero")

        plt.show()


### Inicializa a lista de livros
books = CollectionBook()

In [ ]:
books = CollectionBook()


#### Cadastra Generos padrão

In [ ]:
book_genre_technology_and_programming = BookGenre("Tecnologia / Programação")
book_genre_psychology = BookGenre("Psicologia")
book_genre_fiction = BookGenre("Ficção")
book_genre_history = BookGenre("História")
book_genre_self_help = BookGenre("Autoajuda")
book_genre_science = BookGenre("Ciência")
book_genre_romance = BookGenre("Romance")
book_genre_thriller = BookGenre("Suspense / Thriller")
book_genre_fantasy = BookGenre("Fantasia")
book_genre_biography = BookGenre("Biografia")

#### Cadastra livros padrão

In [ ]:
books.add_book(Book("Código Limpo", "Robert Cecil Martin", book_genre_technology_and_programming, 1))
books.add_book(Book("Refatoração", "Martin Fowler", book_genre_technology_and_programming, 1))
books.add_book(Book("Algoritmos: Teoria e Prática", "Thomas H. Cormen", book_genre_technology_and_programming, 1))
books.add_book(Book("Dream psychology, psychoanalysis for beginners: Psicanálise Para Principiantes", "Sigmund Freud", book_genre_psychology, 1))
books.add_book(Book("O Poder do Hábito", "Charles Duhigg", book_genre_psychology, 1))
books.add_book(Book("Em Busca de Sentido", "Viktor Frankl", book_genre_psychology, 1))
books.add_book(Book("1984", "George Orwell", book_genre_fiction, 1))
books.add_book(Book("Admirável Mundo Novo", "Aldous Huxley", book_genre_fiction, 1))
books.add_book(Book("Sapiens: Uma Breve História da Humanidade", "Yuval Noah Harari", book_genre_history, 1))
books.add_book(Book("Guns, Germs, and Steel", "Jared Diamond", book_genre_history, 1))
books.add_book(Book("O Segredo", "Rhonda Byrne", book_genre_self_help, 1))
books.add_book(Book("Como Fazer Amigos e Influenciar Pessoas", "Dale Carnegie", book_genre_self_help, 1))
books.add_book(Book("Uma Breve História do Tempo", "Stephen Hawking", book_genre_science, 1))
books.add_book(Book("O Gene: Uma História Íntima", "Siddhartha Mukherjee", book_genre_science, 1))
books.add_book(Book("Orgulho e Preconceito", "Jane Austen", book_genre_romance, 1))
books.add_book(Book("Dom Casmurro", "Machado de Assis", book_genre_romance, 1))
books.add_book(Book("O Código Da Vinci", "Dan Brown", book_genre_thriller, 1))
books.add_book(Book("Garota Exemplar", "Gillian Flynn", book_genre_thriller, 1))
books.add_book(Book("Harry Potter e a Pedra Filosofal", "J.K. Rowling", book_genre_fantasy, 1))
books.add_book(Book("Steve Jobs", "Walter Isaacson", book_genre_biography, 1))


#### Lista os livros cadastrados

In [ ]:
print(f"{len(books.list_all_books())} Todos os livros: \n- {"\n- ".join([book.title for book in books.list_all_books()])}")

#### Programa

In [ ]:
from typing import Callable


class ProgramLoop:
    def __init__(self, actions_command: list[tuple[tuple[int, str], Callable[[], None]]]):
        self.actions_command = actions_command
        self.current_command_action: int | None = None
        self.is_running: bool = False
        self.list_command_number_reserved: list[int] = [0, -1]

    def print_command_actions(self):
        for (number, description), _ in self.actions_command:
            print(f"[{number}]: {description}")
        print("[0]: Parar")
        print("[-1]: Voltar")

    def execute_command(self, command_number: int):
        for (number, description), callback in self.actions_command:
            if command_number == number:
                print(f"\n======== {description} =======\n")
                try:
                    callback()
                except Exception:
                    print(f"Voltando para menu principal")
                finally:
                    # sempre volta ao menu
                    self.current_command_action = None
                    print("\n")

    def welcome(self):
        print("\nBem vindo(a), Gestão de livros")
        print("Escolha alguma opção abaixo")
        self.print_command_actions()

    def exist_command_number_from_actions_command(self, command_number: int) -> bool:
        return any(number == command_number for (number, _), _ in self.actions_command)

    def listener_typed_command_number(self) -> int | None:
        try:
            command_number: int = int(input("Escolhe alguma opção: "))
        except ValueError:
            print("Digite apenas números!")
            return None

        if not self.exist_command_number_from_actions_command(command_number) \
           and command_number not in self.list_command_number_reserved:
            print("Opção Inválida")
            return None

        return command_number

    def run(self):
        self.is_running = True

        while self.is_running:
            if self.current_command_action is None:
                self.welcome()
                command_number = self.listener_typed_command_number()
            else:
                command_number = self.current_command_action

            if command_number is None:
                continue

            if command_number == 0:
                self.is_running = False
                self.current_command_action = None
                break

            if command_number == -1:
                self.current_command_action = None
                continue

            self.current_command_action = command_number
            self.execute_command(command_number)


### Funções para as escolhas

In [ ]:
def list_books():
    print("-", "\n- ".join([book.title for book in books.list_all_books()]))

def show_books_dashboard():
    books.show_chart_difference_between_total_of_books_and_genres()
    books.show_chart_about_books_by_genres()

def search_book_by_title():
    book_title: str = str(input("Digite titulo do livro"))

    books_found = books.search_book_by_title(book_title)

    if len(books_found) == 0:
        print("Livro existe")
        return

    print(f"{len(books_found)} Livros encontrados:")
    print("-----------------------------------------")
    print("-", "\n- ".join([book.title for book in books_found]))

def register_book():
    def show_list_genre_with_id():
        print("\nLista de genero textuais existentes:")
        for index, genre in enumerate(books.list_genres()):
            print(f"- [{index + 1}]: {genre}")

    def validate_to_exit(action_value: str):
        if action_value == '-1':
            raise Exception('Voltando para menu principal')

    def input_int(prompt: str, allow_exit: bool = True) -> int:
        while True:
            value = input(prompt)

            # Tratamento para sair
            if allow_exit and value == "-1":
                raise Exception("-1")

            # Verifica se é número válido
            if value.isdigit():
                return int(value)

            print("Valor inválido! Digite apenas números.")

    try:
        book_title: str = str(input("Digite titulo do livro ou -1 para sair"))
        validate_to_exit(book_title)

        book_author: str = str(input("Digite autor do livro ou -1 para sair"))
        validate_to_exit(book_author)

        book_copies: int = input_int("Quantidade de exemplares ou -1 para sair")
        validate_to_exit(str(book_copies))

        show_list_genre_with_id()
        book_genre_input: str = str(input("Novo nome do novo genero textual ou escolhe numerador do livro ou -1 para sair"))
        validate_to_exit(book_genre_input)

        if book_genre_input.isdigit():
            book_genre_index = int(book_genre_input)
            genre_name = books.list_genres()[book_genre_index - 1]
        else:
            genre_name = book_genre_input

        genre_instance = BookGenre(genre_name)

        books.add_book(Book(book_title, book_author, genre_instance, book_copies))

        print('\nLivro registrado com sucesso!')

    except Exception as error:
        print(error)

### Inicialização do programa

In [ ]:
program = ProgramLoop(list([
    ((1, "Novo livro"), lambda: register_book()),
    ((2, "Lista de livros"), lambda: list_books()),
    ((3, "Buscar livro por titulo"), lambda: search_book_by_title()),
    ((4, "Dashboard"), lambda: show_books_dashboard()),
]))

program.run()